In [5]:
import numpy as np
import pandas as pd

In [6]:
train = pd.read_csv('C:\\Users\\Goldie\\Downloads\\ReNew_hack\\train.csv')

In [7]:
target = train['Target']

In [8]:
train['timestamp'] = pd.to_datetime(train['timestamp'])

In [9]:
#train['timestamp'].dt.weekday

In [10]:
train['month'] = train['timestamp'].dt.month
train['day'] = train['timestamp'].dt.day
train['hr'] = train['timestamp'].dt.hour
train['min'] = train['timestamp'].dt.minute
train['weekday'] = train['timestamp'].dt.weekday

In [11]:
train.drop(['timestamp','Target'],axis=1,inplace=True)

In [12]:
list_F = ['active_power_calculated_by_converter', 'active_power_raw',
       'ambient_temperature', 'generator_speed', 'generator_winding_temp_max',
       'grid_power10min_average', 'nc1_inside_temp', 'nacelle_temp',
       'reactice_power_calculated_by_converter', 'reactive_power',
       'wind_direction_raw', 'wind_speed_raw', 'wind_speed_turbulence']

In [13]:
for x in list_F:
    
    train[x] = train[x].apply(lambda t : 0.000001 if t <= 0 else t)
    
    train[x] = np.log(train[x])

In [14]:
#from sklearn.feature_selection import RFECV

In [15]:
#from sklearn.tree import DecisionTreeRegressor

In [16]:
#dtr = DecisionTreeRegressor(random_state=498)

In [17]:
#selector = RFECV(dtr, step=1, cv=5,scoring='neg_mean_absolute_percentage_error',verbose=5,n_jobs=3,min_features_to_select=5)

In [18]:
#selector = selector.fit(train, target)

In [19]:
#list_F2 = list(selector.get_feature_names_out())

In [20]:
#list_F2 = ['active_power_calculated_by_converter', 'ambient_temperature', 'generator_winding_temp_max', 'grid_power10min_average', 'nc1_inside_temp', 'nacelle_temp', 'reactive_power', 'wind_direction_raw']

In [21]:
#train = train.loc[:,list_F2]

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train,target, random_state=578,test_size=0.2,stratify=train['turbine_id'])

In [24]:
encoder_df = pd.DataFrame(X_train['turbine_id'],columns=['turbine_id'])
encoder_df['Target'] = list(y_train)

In [25]:
encoder_dict = encoder_df.groupby('turbine_id')['Target'].mean().to_dict()

In [26]:
def turbine_encoder(e,t):
    
    try:
        
        return e[t]
    
    except:
        
        return 0

In [27]:
X_train['turbine_id'] = X_train['turbine_id'].apply(lambda t : turbine_encoder(encoder_dict,t))
X_test['turbine_id'] = X_test['turbine_id'].apply(lambda t : turbine_encoder(encoder_dict,t))

In [28]:
#from sklearn.preprocessing import FunctionTransformer
#from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import QuantileTransformer

In [29]:
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [30]:
etr = ExtraTreesRegressor(n_estimators=200, random_state=385,n_jobs=3)
rfr = RandomForestRegressor(n_jobs=3,random_state=298)
lgbmr = LGBMRegressor(random_state=120, n_jobs=3)
xgbr = XGBRegressor(n_jobs=3,random_state=476)
cbr = CatBoostRegressor(random_state=386,loss_function='MAPE')

rscv = RandomizedSearchCV(etr,params,random_state=385,n_jobs=3,cv=5,verbose=5,scoring='neg_mean_absolute_percentage_error',n_iter=5)

In [31]:
from sklearn.metrics import mean_absolute_percentage_error

In [32]:
test = pd.read_csv('C:\\Users\\Goldie\\Downloads\\ReNew_hack\\test.csv')

In [33]:
test['timestamp'] = pd.to_datetime(test['timestamp'])

In [34]:
test['turbine_id'] = test['turbine_id'].apply(lambda t : turbine_encoder(encoder_dict,t))

In [35]:
test['month'] = test['timestamp'].dt.month
test['day'] = test['timestamp'].dt.day
test['hr'] = test['timestamp'].dt.hour
test['min'] = test['timestamp'].dt.minute
test['weekday'] = test['timestamp'].dt.weekday

In [36]:
test.drop(['timestamp'],axis=1,inplace=True)

In [37]:
for x in list_F:
    
    test[x] = test[x].apply(lambda t : 0.000001 if t <= 0 else t)
    
    test[x] = np.log(test[x])

In [38]:
est_preds = []

for x,y in zip([etr,rfr,lgbmr,xgbr,cbr],['etr','rfr','lgbmr','xgbr','cbr']):
    
    x.fit(X_train, y_train)
    
    print('{} has test mape of {}'.format(y,mean_absolute_percentage_error(y_test, x.predict(X_test))))
    
    est_preds.append(x.predict(test))    
    

etr has test mape of 0.003210077833646437
rfr has test mape of 0.004991821457811534
lgbmr has test mape of 0.014266849858556443


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


xgbr has test mape of 0.012501590702402367


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0:	learn: 0.0405198	total: 253ms	remaining: 4m 12s
1:	learn: 0.0400807	total: 334ms	remaining: 2m 46s
2:	learn: 0.0396493	total: 422ms	remaining: 2m 20s
3:	learn: 0.0392499	total: 518ms	remaining: 2m 8s
4:	learn: 0.0388440	total: 609ms	remaining: 2m 1s
5:	learn: 0.0384477	total: 706ms	remaining: 1m 56s
6:	learn: 0.0380655	total: 804ms	remaining: 1m 54s
7:	learn: 0.0376925	total: 904ms	remaining: 1m 52s
8:	learn: 0.0373473	total: 1.01s	remaining: 1m 50s
9:	learn: 0.0369991	total: 1.1s	remaining: 1m 49s
10:	learn: 0.0366619	total: 1.2s	remaining: 1m 48s
11:	learn: 0.0363342	total: 1.3s	remaining: 1m 46s
12:	learn: 0.0360237	total: 1.39s	remaining: 1m 45s
13:	learn: 0.0356728	total: 1.49s	remaining: 1m 44s
14:	learn: 0.0353711	total: 1.58s	remaining: 1m 44s
15:	learn: 0.0350765	total: 1.69s	remaining: 1m 43s
16:	learn: 0.0347958	total: 1.79s	remaining: 1m 43s
17:	learn: 0.0344816	total: 1.9s	remaining: 1m 43s
18:	learn: 0.0341970	total: 2s	remaining: 1m 43s
19:	learn: 0.0339267	total: 2.1

...

316:	learn: 0.0175027	total: 29.5s	remaining: 1m 3s
317:	learn: 0.0174948	total: 29.6s	remaining: 1m 3s
318:	learn: 0.0174860	total: 29.7s	remaining: 1m 3s
319:	learn: 0.0174755	total: 29.8s	remaining: 1m 3s
320:	learn: 0.0174693	total: 29.8s	remaining: 1m 3s
321:	learn: 0.0174604	total: 29.9s	remaining: 1m 3s
322:	learn: 0.0174511	total: 30s	remaining: 1m 2s
323:	learn: 0.0174444	total: 30.1s	remaining: 1m 2s
324:	learn: 0.0174335	total: 30.2s	remaining: 1m 2s
325:	learn: 0.0174242	total: 30.3s	remaining: 1m 2s
326:	learn: 0.0174103	total: 30.4s	remaining: 1m 2s
327:	learn: 0.0174043	total: 30.5s	remaining: 1m 2s
328:	learn: 0.0173949	total: 30.6s	remaining: 1m 2s
329:	learn: 0.0173902	total: 30.7s	remaining: 1m 2s
330:	learn: 0.0173820	total: 30.7s	remaining: 1m 2s
331:	learn: 0.0173759	total: 30.8s	remaining: 1m 2s
332:	learn: 0.0173707	total: 30.9s	remaining: 1m 1s
333:	learn: 0.0173632	total: 31s	remaining: 1m 1s
334:	learn: 0.0173561	total: 31.1s	remaining: 1m 1s
335:	learn: 0.01

476:	learn: 0.0163896	total: 43.8s	remaining: 48s
477:	learn: 0.0163826	total: 43.9s	remaining: 47.9s
478:	learn: 0.0163779	total: 44s	remaining: 47.8s
479:	learn: 0.0163717	total: 44.1s	remaining: 47.7s
480:	learn: 0.0163656	total: 44.2s	remaining: 47.7s
481:	learn: 0.0163618	total: 44.2s	remaining: 47.6s
482:	learn: 0.0163542	total: 44.3s	remaining: 47.5s
483:	learn: 0.0163482	total: 44.4s	remaining: 47.4s
484:	learn: 0.0163439	total: 44.5s	remaining: 47.3s
485:	learn: 0.0163398	total: 44.6s	remaining: 47.2s
486:	learn: 0.0163307	total: 44.7s	remaining: 47.1s
487:	learn: 0.0163248	total: 44.8s	remaining: 47s
488:	learn: 0.0163195	total: 44.9s	remaining: 46.9s
489:	learn: 0.0163159	total: 45s	remaining: 46.8s
490:	learn: 0.0163089	total: 45.1s	remaining: 46.7s
491:	learn: 0.0163034	total: 45.1s	remaining: 46.6s
492:	learn: 0.0162949	total: 45.2s	remaining: 46.5s
493:	learn: 0.0162906	total: 45.3s	remaining: 46.4s
494:	learn: 0.0162847	total: 45.4s	remaining: 46.3s
495:	learn: 0.016278

637:	learn: 0.0155951	total: 58.6s	remaining: 33.2s
638:	learn: 0.0155912	total: 58.7s	remaining: 33.1s
639:	learn: 0.0155867	total: 58.7s	remaining: 33s
640:	learn: 0.0155840	total: 58.8s	remaining: 32.9s
641:	learn: 0.0155797	total: 58.9s	remaining: 32.9s
642:	learn: 0.0155766	total: 59s	remaining: 32.8s
643:	learn: 0.0155741	total: 59.1s	remaining: 32.7s
644:	learn: 0.0155710	total: 59.2s	remaining: 32.6s
645:	learn: 0.0155680	total: 59.3s	remaining: 32.5s
646:	learn: 0.0155586	total: 59.4s	remaining: 32.4s
647:	learn: 0.0155559	total: 59.4s	remaining: 32.3s
648:	learn: 0.0155526	total: 59.5s	remaining: 32.2s
649:	learn: 0.0155477	total: 59.6s	remaining: 32.1s
650:	learn: 0.0155451	total: 59.7s	remaining: 32s
651:	learn: 0.0155419	total: 59.8s	remaining: 31.9s
652:	learn: 0.0155387	total: 59.9s	remaining: 31.8s
653:	learn: 0.0155353	total: 60s	remaining: 31.7s
654:	learn: 0.0155315	total: 1m	remaining: 31.6s
655:	learn: 0.0155281	total: 1m	remaining: 31.5s
656:	learn: 0.0155256	tota

796:	learn: 0.0150133	total: 1m 12s	remaining: 18.6s
797:	learn: 0.0150102	total: 1m 13s	remaining: 18.5s
798:	learn: 0.0150071	total: 1m 13s	remaining: 18.4s
799:	learn: 0.0150026	total: 1m 13s	remaining: 18.3s
800:	learn: 0.0149999	total: 1m 13s	remaining: 18.2s
801:	learn: 0.0149961	total: 1m 13s	remaining: 18.1s
802:	learn: 0.0149938	total: 1m 13s	remaining: 18s
803:	learn: 0.0149911	total: 1m 13s	remaining: 18s
804:	learn: 0.0149888	total: 1m 13s	remaining: 17.9s
805:	learn: 0.0149843	total: 1m 13s	remaining: 17.8s
806:	learn: 0.0149816	total: 1m 13s	remaining: 17.7s
807:	learn: 0.0149788	total: 1m 14s	remaining: 17.6s
808:	learn: 0.0149755	total: 1m 14s	remaining: 17.5s
809:	learn: 0.0149721	total: 1m 14s	remaining: 17.4s
810:	learn: 0.0149700	total: 1m 14s	remaining: 17.3s
811:	learn: 0.0149659	total: 1m 14s	remaining: 17.2s
812:	learn: 0.0149635	total: 1m 14s	remaining: 17.1s
813:	learn: 0.0149609	total: 1m 14s	remaining: 17s
814:	learn: 0.0149581	total: 1m 14s	remaining: 16.9s

952:	learn: 0.0145924	total: 1m 27s	remaining: 4.29s
953:	learn: 0.0145889	total: 1m 27s	remaining: 4.2s
954:	learn: 0.0145875	total: 1m 27s	remaining: 4.11s
955:	learn: 0.0145854	total: 1m 27s	remaining: 4.02s
956:	learn: 0.0145841	total: 1m 27s	remaining: 3.93s
957:	learn: 0.0145814	total: 1m 27s	remaining: 3.84s
958:	learn: 0.0145799	total: 1m 27s	remaining: 3.74s
959:	learn: 0.0145779	total: 1m 27s	remaining: 3.65s
960:	learn: 0.0145766	total: 1m 27s	remaining: 3.56s
961:	learn: 0.0145745	total: 1m 27s	remaining: 3.47s
962:	learn: 0.0145724	total: 1m 27s	remaining: 3.38s
963:	learn: 0.0145707	total: 1m 28s	remaining: 3.29s
964:	learn: 0.0145676	total: 1m 28s	remaining: 3.2s
965:	learn: 0.0145659	total: 1m 28s	remaining: 3.1s
966:	learn: 0.0145638	total: 1m 28s	remaining: 3.01s
967:	learn: 0.0145624	total: 1m 28s	remaining: 2.92s
968:	learn: 0.0145611	total: 1m 28s	remaining: 2.83s
969:	learn: 0.0145592	total: 1m 28s	remaining: 2.74s
970:	learn: 0.0145573	total: 1m 28s	remaining: 2.

In [3]:
#etr.fit(X_train, y_train)

In [4]:
#mean_absolute_percentage_error(y_test, etr.predict(X_test))

In [ ]:
#log transform - 0.003568948955724395
#Power transform(yeo-johnson) - 0.005723019040177586
#Power transform(box-cox) - 0.005872130643571877

In [39]:
import gc
#gc.collect()

In [40]:
del train,X_train,y_train,X_test,y_test

In [41]:
gc.collect()

45

In [ ]:
#test = test.loc[:,list_F2]

In [45]:
#preds = etr.predict(test)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:    3.1s
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:   25.4s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:  1.3min
[Parallel(n_jobs=3)]: Done 200 out of 200 | elapsed:  1.7min finished


In [42]:
final = pd.DataFrame(est_preds[0],columns=['etr'])
for x,y in zip(est_preds[1:],['rfr','lgbmr','xgbr','cbr']):
    
    final[y] = list(x)    
    

In [43]:
final

,etr,rfr,lgbmr,xgbr,cbr
0,50.022884,49.990808,50.231422,50.142494,50.034911
1,41.058106,41.083206,41.554580,41.078846,41.525909
2,48.319249,48.363951,48.964794,49.119507,49.104248
3,45.945999,44.818389,45.854303,45.723408,45.758566
4,45.316322,45.286491,45.662812,45.601398,45.534520
...,...,...,...,...,...
303197,50.997879,50.812749,50.492820,50.305420,50.655129
303198,47.636265,47.786982,47.816507,47.712223,47.692769
303199,46.113607,45.997301,46.398580,46.899246,46.287322
303200,47.186985,47.212499,46.617627,46.800152,46.735872


In [44]:
final['Target'] = final.median(axis=1)

In [45]:
final

,etr,rfr,lgbmr,xgbr,cbr,Target
0,50.022884,49.990808,50.231422,50.142494,50.034911,50.034911
1,41.058106,41.083206,41.554580,41.078846,41.525909,41.083206
2,48.319249,48.363951,48.964794,49.119507,49.104248,48.964794
3,45.945999,44.818389,45.854303,45.723408,45.758566,45.758566
4,45.316322,45.286491,45.662812,45.601398,45.534520,45.534520
...,...,...,...,...,...,...
303197,50.997879,50.812749,50.492820,50.305420,50.655129,50.655129
303198,47.636265,47.786982,47.816507,47.712223,47.692769,47.712223
303199,46.113607,45.997301,46.398580,46.899246,46.287322,46.287322
303200,47.186985,47.212499,46.617627,46.800152,46.735872,46.800152


In [46]:
predictions = pd.DataFrame(list(final['Target']),columns=['Target'])

In [47]:
predictions.to_csv('C:\\Users\\Goldie\\Downloads\\ReNew_hack\\Renew_ETR_Tx_Log_Blend.csv',index=False)

In [ ]:
#150 - 0.0032496540200138077
#160 - 0.0032379160745969397
#180 - 0.003214216515975729